title: Membership Home v1    
author: Fabio Schmidt-Fischbach   
date: 2020-08-17   
region: EU   
summary: Launched in CW 30 2020 (not split-tested). Main idea. Move account benefits to more central position in app. Goal. Increase perceived value of product by gathering all relevant information in one place. Do more people go to explore since we launched Membership Home? No. Those that visit it, are they more engaged? Possibly. From 20% of explore visitors engaging with the content, we currently are currently close to 30%.  What do they look at on explore? 25% of interactions happen with tier benefit feature. Did this also increase the % of premium users that ever looked at these screens? Yes, prior launch roughly 1% of all premium logins went to the premium benefit overview screen : this rose by roughly 1pp. Did it move the needle in terms of cancellation/cs contacts? No way to tell since this was not split test released. 
link: https://docs.google.com/presentation/d/1aeA10Q0V0vf6H78U6fznAzYjUWYf_d8mG0BA6P3IxCY/edit?usp=sharing    
tags: membership, membership home, explore, inapp, engage, benefits, allianz

In [1]:
import pandas as pd
import os
import numpy as np
import altair as alt
import psycopg2
import psycopg2.extras
import math
import time

/Users/fabioschmidt-fischbach/anaconda3/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [15]:
# Pulling the data.

In [2]:
def exponential_backoff(fn):
    """Exponential backoff upon Error."""

    def wrapper(*args, **kwargs):
        i = 0
        while i < 5:
            try:
                time.sleep(1)
                print("Calling DB.")
                data = fn(*args, **kwargs)
                return data
            except Exception as e:
                # Exponential backoff in case of rate limiting
                sleep_int = math.pow(4, i + 2)
                print(f"Sleeping {sleep_int} seconds! Because of:")
                print(e)
                time.sleep(sleep_int)
                print(f"Slept {sleep_int} seconds!")
                i += 1
        return None

    return wrapper


def connect_redshift():
    dbname = "n26"
    host = "n26-dwh.cfxsmcyyfcch.eu-central-1.redshift.amazonaws.com"
    pwd = ""
    username = "fabio_schmidtfischbach_reader"
    try:
        conn = psycopg2.connect(
            host=host,
            user=username,
            port=5439,
            password=pwd,
            dbname=dbname,
            options="-c statement_timeout=500000",
        )
    except psycopg2.OperationalError as e:
        print("Unable to connect!\n{}".format(e))
        raise

    return conn


def cursor_redshift(conn):
    return conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor)


@exponential_backoff
def query_to_df(query):
    conn = connect_redshift()
    cur = cursor_redshift(conn)
    try:
        cur.execute(query)
        result = cur.fetchall()
        conn.commit()
        conn.close()
    except Exception:
        conn.rollback()
        conn.close()
        raise

    result = pd.DataFrame(result)
    return result

In [ ]:
query = """
select cmd.id, se_label, se_property, collector_date, count(1)  
from dbt.snowplow 
inner join cmd_users as cmd using (user_created)
where event_type in (334497123, -52899623, 610, 616) and collector_date >= '2020-07-01'
group by 1,2,3,4
order by 1,2,3,4
"""
df = query_to_df(query)

df.to_csv("explore_events.csv")
df.head()

In [ ]:
query = """

create TEMPORARY table all_logins as
select cmd.id, login_date::date as login_date, zup.product_id, row_number() over() as rn 
from dbt.zrh_login_day as zld 
inner join cmd_users as cmd on zld.user_created = cmd.user_created 
left join dbt.zrh_user_product as zup on zup.user_created = zld.user_created and login_date between subscription_valid_from and subscription_valid_until 
where login_date >= '2020-07-01' ;


select * from all_logins where rn <= 9000000;
select * from all_logins where rn > 9000000 and rn <= 17000000;
select * from all_logins where rn > 17000000 and rn <= 26000000;

"""

In [ ]:
query = """
select collector_date, cmd.id, 
		se_action, 
		se_label, 
		se_property, 
	   count(1) 
from dbt.snowplow 
inner join cmd_users as cmd using (user_created)
where se_action in ('membership.benefits-list.view',
					'premium.membership_insurance_coverage_detail_screen.viewed',
					'premium.membership_insurance_coverage_screen.viewed',
					'membership.atm_usage.page_viewed', 
					'premium.membership_overview_screen.viewed') and collector_date >= '2020-07-01' 
group by 1,2,3,4,5 order by 1 
"""

In [ ]:
query = """

with cs as ( 
select user_id, count(1) as cs_contacts, min(initiated_date) as cs_ts 
from dbt.sf_all_contacts 
where case_tag in ('61 You', '61 black', '64 Business You', '64 business black', '62 metal') and c_level_report = True 
group by 1 
),
cancellation_revocation as ( 

select cmd_users.id, min(updated_at) as drop_out
from lp_user_product 
inner join cmd_users using (user_created) 
where product_id in ('BLACK_CARD_MONTHLY','METAL_CARD_MONTHLY','BUSINESS_BLACK','BUSINESS_METAL') 
	and subscription_valid_from >= '2020-07-01'
	and status in ('TO_BE_CANCELLED', 'REVOKED')
group by 1 
)

select zu.user_id, drop_out, kyc_first_completed, product_id , cs_ts, cs_contacts
from dbt.zrh_users as zu 
left join cancellation_revocation as cr on cr.id = zu.user_id 
left join cs on cs.user_id = zu.user_id 
where kyc_first_completed >= '2020-07-01'

"""

In [38]:
login = pd.read_csv("login_events_1.csv")
login.shape
login2 = pd.read_csv("login_events_2.csv")
login2.shape
login3 = pd.read_csv("login_events_3.csv")
login3.shape

login = login.append(login2)
login = login.append(login3)

login.to_csv("login_events_final.csv")

### Do we see a rise in interest for the explore tab? 

In [9]:
df = pd.read_csv("explore_events.csv")

df = df.loc[df["se_label"] == "view", :]

df = df.groupby(["collector_date"])["id"].agg("nunique").reset_index()
df["collector_date"] = df["collector_date"].astype(str)

alt.Chart(df).mark_line().encode(
    x=alt.X("collector_date"), y=alt.Y("id:Q", axis=alt.Axis(title="Number of users"))
).properties(width=600, height=400, title="Number of users that visit explore")

alt.Chart(...)

In [10]:
df = pd.read_csv("login_events_final.csv")

df.head()

explore = pd.read_csv("explore_events.csv")

explore = explore.loc[explore["se_label"] == "view", :]
explore = (
    explore.groupby(["id", "collector_date"])["se_label"].agg("count").reset_index()
)

df = df.merge(
    explore, left_on=["id", "login_date"], right_on=["id", "collector_date"], how="left"
)

df["explore"] = 0
df.loc[df["se_label"].isna() == False, "explore"] = 1

# compute % of users that login and also visit explore.
df = df.groupby(["login_date"])["explore"].agg("mean").reset_index()

alt.Chart(df).mark_line().encode(
    x=alt.X("login_date"),
    y=alt.Y(
        "explore:Q", axis=alt.Axis(format="%", title="% of logins that visit explore")
    ),
).properties(width=600, height=400, title="% of logins that visit explore")

alt.Chart(...)

In [83]:
df = pd.read_csv("login_events_final.csv")

explore = pd.read_csv("explore_events.csv")

explore = explore.loc[explore["se_label"] == "view", :]
explore = (
    explore.groupby(["id", "collector_date"])["se_label"].agg("count").reset_index()
)

df = df.merge(
    explore, left_on=["id", "login_date"], right_on=["id", "collector_date"], how="left"
)

df["explore"] = 0
df.loc[df["se_label"].isna() == False, "explore"] = 1

# compute % of users that login and also visit explore.
df = df.groupby(["login_date", "product_id"])["explore"].agg("mean").reset_index()

alt.Chart(df).mark_line().encode(
    x=alt.X("login_date"),
    y=alt.Y(
        "explore:Q", axis=alt.Axis(format="%", title="% of logins that visit explore")
    ),
    color="product_id:N",
).properties(width=600, height=400, title="% of logins that visit explore by product.")

alt.Chart(...)

In [77]:
df = pd.read_csv("login_events_final.csv")

df.head()

explore = pd.read_csv("explore_events.csv")

explore = (
    explore.groupby(["id", "collector_date"])["se_label"].agg("count").reset_index()
)

df = df.merge(
    explore,
    left_on=["id", "login_date"],
    right_on=["id", "collector_date"],
    how="inner",
)

# compute % of users that login and also visit explore.
df = df.groupby(["login_date", "product_id"])["se_label"].agg("mean").reset_index()

alt.Chart(df).mark_line().encode(
    x=alt.X("login_date"),
    y=alt.Y(
        "se_label:Q",
        axis=alt.Axis(title="Avg. number of explore interactions per visit"),
    ),
    color="product_id:N",
).properties(
    width=600, height=400, title="Avg. number of explore interactions per visit."
)

alt.Chart(...)

In [82]:
df = pd.read_csv("login_events_final.csv")

df.head()

explore = pd.read_csv("explore_events.csv")

explore["interaction"] = 0
explore.loc[explore["se_property"].isna() == False, "interaction"] = 1

explore = (
    explore.groupby(["id", "collector_date"])["interaction"].agg("max").reset_index()
)

explore = explore.groupby(["collector_date"])["interaction"].agg("mean").reset_index()

alt.Chart(explore).mark_line().encode(
    x=alt.X("collector_date"),
    y=alt.Y(
        "interaction:Q",
        axis=alt.Axis(format="%", title="% visits to explore with interaction"),
    ),
).properties(width=600, height=400, title="% visits to explore with interaction")

alt.Chart(...)

In [85]:
import re

df = pd.read_csv("explore_events.csv")

df = df.loc[(df["se_label"] == "click"), :]
dictionary = dict()

for value in df["se_property"].unique():
    try:
        if value.upper() == value:
            dictionary[value] = "Partnership"
        elif (
            bool(re.search("insurance", value)) == True
            or bool(re.search("credit", value)) == True
            or bool(re.search("overdraft", value)) == True
            or bool(re.search("fixedterm", value)) == True
        ):
            dictionary[value] = "Bank products"
        elif value in (
            "N26 Business Metal",
            "N26 Business You",
            "N26 Metal",
            "N26 You",
        ):
            dictionary[value] = "Upgrade"
        else:
            dictionary[value] = value
    except:
        print(value)

# remap values
df["destination"] = df["se_property"].map(dictionary)

df = df.groupby(["collector_date", "destination"])["id"].agg("nunique").reset_index()
df["perc"] = df["id"] / df.groupby("collector_date")["id"].transform("sum")

alt.Chart(df).mark_line().encode(
    x=alt.X("collector_date"),
    y=alt.Y(
        "perc:Q",
        axis=alt.Axis(format="%", title="% of users that do anything on explore"),
        scale=alt.Scale(base=10),
    ),
    color="destination:N",
).properties(width=400, height=400, title="% of users that do anything on explore")

nan


alt.Chart(...)

In [105]:
import re

df = pd.read_csv("explore_events.csv")

df = df.loc[(df["se_label"] == "click"), :]
dictionary = dict()

for value in df["se_property"].unique():
    try:
        if value.upper() == value:
            dictionary[value] = "Partnership"
        elif (
            bool(re.search("insurance", value)) == True
            or bool(re.search("credit", value)) == True
            or bool(re.search("overdraft", value)) == True
            or bool(re.search("fixedterm", value)) == True
        ):
            dictionary[value] = "Bank products"
        elif value in (
            "N26 Business Metal",
            "N26 Business You",
            "N26 Metal",
            "N26 You",
        ):
            dictionary[value] = "Upgrade"
        else:
            dictionary[value] = value
    except:
        print(value)

# remap values
df["destination"] = df["se_property"].map(dictionary)
logins = pd.read_csv("login_events_final.csv")

df = df.merge(
    logins, left_on=["collector_date", "id"], right_on=["login_date", "id"], how="left"
)

df = (
    df.groupby(["collector_date", "destination", "product_id"])["id"]
    .agg("nunique")
    .reset_index()
)
df["perc"] = df["id"] / df.groupby(["collector_date", "product_id"])["id"].transform(
    "sum"
)

df = df.loc[
    df["product_id"].isin(["BLACK_CARD_MONTHLY", "METAL_CARD_MONTHLY", "STANDARD"]), :
]

alt.Chart(df).mark_line().encode(
    x=alt.X("collector_date"),
    y=alt.Y(
        "perc:Q",
        axis=alt.Axis(format="%", title="% of users that do anything on explore"),
        scale=alt.Scale(base=10),
    ),
    color="destination:N",
).properties(
    width=400, height=200, title="% of users that do anything on explore"
).facet(
    facet="product_id", columns=3
)

nan


alt.FacetChart(...)

In [104]:
df = pd.read_csv("benefit_details.csv")

df.loc[
    df["se_action"] == "premium.membership_benefits_screen.viewed", "se_action"
] = "membership.benefits-list.view"

df = df.loc[df["se_action"] == "membership.benefits-list.view", :]

df = df.groupby(["collector_date", "se_action", "id"])["count"].agg("sum").reset_index()

logins = pd.read_csv("login_events_final.csv")

df = logins.merge(
    df, right_on=["collector_date", "id"], left_on=["login_date", "id"], how="left"
)

df["count"] = df["count"].fillna(0)

df["look_at_benefits"] = 0
df.loc[df["count"] > 0, "look_at_benefits"] = 1

df = (
    df.groupby(["login_date", "product_id"])["look_at_benefits"]
    .agg("mean")
    .reset_index()
)

alt.Chart(df).mark_line().encode(
    x=alt.X("login_date"),
    y=alt.Y(
        "look_at_benefits:Q",
        axis=alt.Axis(format="%", title="% of logins that look at benefits list"),
        scale=alt.Scale(base=10),
    ),
    color="product_id:N",
).properties(width=400, height=400, title="% of logins that look at benefits list")

alt.Chart(...)